# Diabetic Retinopathy Detection Using Deep Learning
<img src='https://www.researchgate.net/profile/Srinivasa-Rao-Avanapu/publication/282609747/figure/fig2/AS:281548759814145@1444137863086/Difference-between-Normal-Retina-and-Diabetic-Retinopathy.png'>

# Import Libraries

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,MaxPool2D,Conv2D,Flatten,Dropout

import numpy as np
import pandas as pd

# Create Data Generator for Data Augmentation
* Data was split to train and test .1 to .9 
* Read the data in training generator 
* Read the data in testing generator
* Rescale images for easy trainig and faster convergence
* Apply some augmentation methods such as shear and zoom
* Select class mode categorical
* The target size of images is( 224* 224 ) 
* Batch size is 32 
* All this numbers are trial and error numbers you can change them or add more methods for augmentation.
* I selected this number to save memmory (this is an order from my universty for graduation project)

In [ ]:
# Generating images for the Training set
train_datagen = ImageDataGenerator(
    
rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True
   
    )

training_set = train_datagen.flow_from_directory('../input/drdata/data/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_datagen = ImageDataGenerator(rescale = 1./255)

test_set = test_datagen.flow_from_directory('../input/drdata/data/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')



# Building Our Model
* Using pretrained model mobilenet-V2 
* I selected average pooling 
* Add 2 hidden layers for trainig and 1 for output 
* Compile the model 
* Only 10 epochs for training 
* Early stop if the model stop learning
* Return to the best weights which model reached

In [ ]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False


inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(5, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy',tf.keras.metrics.Recall()]
)

history = model.fit(
    training_set,
    validation_data=test_set,
    batch_size = 32,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

# Visualize Result 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Retrieve a list of list results on training and validation data
# sets for each training epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc))

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')

plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')


In [ ]:
# dump the pipeline model
model.save("model_num.h5")